In [1]:
import sys

In [2]:
!{sys.executable} -m pip install itables

You should consider upgrading via the '/Users/fkraeutli/anaconda/bin/python -m pip install --upgrade pip' command.


In [3]:
from itables import show
from lxml import etree
import pandas as pd

<IPython.core.display.Javascript object>

In [4]:
inputFile = '/Users/fkraeutli/Sites/bso-curation-data/data/nb/source/WMC_Records_20201201.xml'
outputFile = './output/nb-analysis.html'
output = ''

In [5]:
root = etree.parse(inputFile)

In [6]:
records = root.findall("/Record")

Look at number of records

In [7]:
print("Found %d records" % len(records))

Found 8384 records


Parse XML file

In [8]:
def addDataElements(elements, newElements):
    for element in newElements:
        id = element.get("ElementId")
        if not id in elements:
            elements[id] = {
                "id": id,
                "name": element.get("ElementName"),
                "type": element.get("ElementType"),
                "typeId": element.get("ElementTypeId"),
                "recordCount": 1
            }
        else:
            elements[id]["recordCount"] += 1
    return elements

dataElements = {}
descriptors = {}
for record in records:
    recordDataElements = record.findall("DetailData/DataElement")
    dataElements = addDataElements(dataElements, recordDataElements)
            
    recordDescriptors = record.findall("Descriptors/Descriptor")
    for descriptor in recordDescriptors:
        id = descriptor.find("Thesaurus").text
        if not id in descriptors:
            descriptors[id] = {
                "id": id,
                "recordCount": 0,
                "hasDetailData": False,
                "dataElements": {}
            }
        else:
            descriptors[id]["recordCount"] += 1
        
        if descriptor.find("DetailData"): 
            descriptors[id]["hasDetailData"] = True
            descriptors[id]["dataElements"] = addDataElements(descriptors[id]["dataElements"], descriptor.findall("DetailData/DataElement"))
            
            

/Users/fkraeutli/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:35: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


## Data elements

In [9]:
data = [[d['id'], d['name'], d['type'], d['recordCount'], d['recordCount']/len(records)] for d in dataElements.values()]
dataElementsAnalysis = pd.DataFrame(data, columns=["Id", "Name", "Type", "Count", "Coverage"])
show(dataElementsAnalysis)

Id,Name,Type,Count,Coverage


In [ ]:
output += "<h2>Data Elements</h2>"
output += dataElementsAnalysis.to_html()

## Descriptors

In [10]:
data = [[d['id'], d['hasDetailData'], d['recordCount'], d['recordCount']/len(records)] for d in descriptors.values()]
descriptorsAnalysis = pd.DataFrame(data, columns=["Id", "Details", "Count", "Coverage"])
show(descriptorsAnalysis)

Id,Details,Count,Coverage


In [ ]:
output += "<h2>Descriptors</h2>"
output += descriptorsAnalysis.to_html()

### Data Elements within Descriptors

In [ ]:
output += "<h3>Data Elements within Descriptors</h3>"

In [11]:
for descriptor in [descriptors[d] for d in descriptors.keys() if descriptors[d]['hasDetailData'] is True]:
    print(descriptor['id'])
    data = [[d['id'], d['name'], d['type'], d['recordCount']] for d in descriptor['dataElements'].values()]
    dataElementsAnalysis = pd.DataFrame(data, columns=["Id", "Name", "Type", "Count"])
    show(dataElementsAnalysis)
    
    output += "<h4>%s</h4>" % descriptor['id']
    output += dataElementsAnalysis.to_html()

Personen


Id,Name,Type,Count


Geografika


Id,Name,Type,Count


Körperschaften


Id,Name,Type,Count


## Tags

In [12]:
tags = list(set([t.tag for t in root.findall('//*')]))
tags.sort()

List all tags

In [13]:
print("\n".join(tags))

AdministrativeData
CreatedOn
DataElement
DateRange
Description
Descriptor
Descriptors
DetailData
ElementValue
FileValue
FromApproxIndicator
FromDate
IdName
IntValue
LastEditedOn
Name
Record
Reference
ReferenceName
References
SeeAlso
TextRepresentation
TextValue
Thesaurus
ToApproxIndicator
ToDate


In [ ]:
output += "<h2>Tags</h2>"
output += "<ul><li>" + "</li><li>".join(tags)